<a href="https://colab.research.google.com/github/amalvarezme/AprendizajeMaquina/blob/main/7_TopicosAvanzados/2_Autoencoders/3_Autoencoder_TwoOutputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import fashion_mnist, mnist
from tensorflow.keras.losses import SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
import numpy as np


# Load and prepare the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()#fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


# create training, validation, and testing sets
x_val = x_train[50000:]
y_val = y_train[50000:]
x_train = x_train[:50000]
y_train = y_train[:50000]
x_train = x_train[..., tf.newaxis]
x_val = x_val[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


In [ ]:
print(x_train.shape,x_val.shape,x_test.shape,y_train.shape,y_val.shape,y_test.shape)

In [ ]:
import matplotlib.pyplot as plt

#plot original images vs reconstructed images
def plot_mnist_autoencoder(x,xpred,cmap='gray',vmin=0,vmax=1):
  fig,ax = plt.subplots(2,x.shape[0],figsize=(8,1))
  for i,class_ in enumerate(range(x.shape[0])):
        ax[0,i].imshow(x[i],cmap=cmap,vmin=vmin,vmax=vmax)
        ax[0,i].set_xticks([])
        ax[0,i].set_yticks([])

        ax[1,i].imshow(xpred[i],cmap=cmap,vmin=vmin,vmax=vmax)
        ax[1,i].set_xticks([])
        ax[1,i].set_yticks([])
  plt.show()
  return

plot_mnist_autoencoder(x_train[:15],x_train[:15])

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

#plot images on latent space
def plot_mnist_2d(Z,y,images,img_w=28,img_h=28,zoom=0.5,cmap='jet'):
    fig, ax = plt.subplots(figsize=(5,5))
    plt.axis('off')
    for i in range(Z.shape[0]):
        #print('img',i+1,'/',Z.shape[0])
        image = images[i].reshape((img_w, img_h))
        im = OffsetImage(image, zoom=zoom,cmap=cmap)
        ab = AnnotationBbox(im, (Z[i,0], Z[i,1]), xycoords='data', frameon=False)
        ax.add_artist(ab)
        ax.update_datalim([(Z[i,0], Z[i,1])])
        ax.autoscale()
    plt.show()

In [ ]:
from sklearn.decomposition import PCA

#traditional PCA algorithm
red = PCA(n_components=2, random_state=123)
Z = red.fit_transform(x_train.reshape(x_train.shape[0],-1))
N = 500
plot_mnist_2d(Z[:N],y_train[:N],x_train[:N],img_w=28,img_h=28,zoom=0.3,cmap='gray')

In [ ]:
# Define the loss object and the optimizer
tf.keras.backend.clear_session()

# Define the autoencoder model
input_img = Input(shape=(28, 28, 1))

# Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
xe = Conv2D(8, (3, 3), activation='relu', padding='same')(x)

# Decoder
x = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(xe)
x = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
reconstructed_img = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Classification branch
x = Flatten(name='fencoded')(xe)
classification_output = Dense(10, activation='softmax')(x)

# Define the model with two outputs
autoencoder = Model(inputs=input_img, outputs=[reconstructed_img, classification_output])

autoencoder.summary()
tf.keras.utils.plot_model(autoencoder)

In [ ]:
# Custom loss function
def custom_loss(lambda_=0.5):
    def custom_loss_autoencoder(y_true, y_pred):
        reconstruction_loss = MeanSquaredError()(y_true[0], y_pred[0])
        classification_loss = SparseCategoricalCrossentropy()(y_true[1], y_pred[1])
        return lambda_*reconstruction_loss + (1-lambda_)*classification_loss
    return custom_loss_autoencoder
# Compile the model
lam_ = 0.25 #reconstruction error weight
autoencoder.compile(optimizer=Adam(), loss=custom_loss(lambda_=lam_))




In [ ]:
# Custom training loop
batch_size = 64
epochs = 20
N = 500
red = PCA(n_components=2, random_state=123)

for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    for x_batch, y_batch in tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size=1024).batch(batch_size):
        with tf.GradientTape() as tape:
            reconstruction, classification = autoencoder(x_batch, training=True)
            loss = autoencoder.loss([x_batch, y_batch], [reconstruction, classification])
            gradients = tape.gradient(loss, autoencoder.trainable_variables)
        autoencoder.optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))

    #val
    loss_ = []
    for x_val_batch, y_val_batch in tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(buffer_size=128).batch(batch_size):
        val_reconstruction, val_classification = autoencoder(x_val_batch, training=False)
        loss_.append(autoencoder.loss([x_val_batch, y_val_batch], [val_reconstruction, val_classification]))
    print(f'Loss: {loss.numpy()} Val_loss: {np.array(loss_).mean()}')
    if (epoch+1)%5 == 0:
      # Test the model
      #plot reconstruction
      plot_mnist_autoencoder(x_val_batch,val_reconstruction)
      #plot encoded space
      encoder_ = tf.keras.Model(inputs=autoencoder.inputs,outputs=autoencoder.get_layer('fencoded').output)
      Z = red.fit_transform(encoder_(x_val))

      plot_mnist_2d(Z[:N],y_val[:N],x_val[:N],img_w=28,img_h=28,zoom=0.3,cmap='gray')



print('done')